In [3]:
import requests
import json

import os
import base64

In [4]:
def elastic_request(data=None, method=None, url=None):
    if method == None:
        method = requests.get
    if data:
        data = json.dumps(data)    

    return method(f"https://192.168.1.153:9200/{url}",
                  headers={
                    "Content-Type": "application/json",
                    "Accept": "application/json",
                    "Authorization": "ApiKey NzhQRTdKTUJPSHd1QnVDSjk2ekE6VTFRdW4zZHdSM3U3cXFHejRRWkNEZw=="
                  },
                  verify=False,
                  data=data)

In [ ]:
rslt = elastic_request(url="players-handbook/_search",
                       data={
                            "query": {
                                "match": {
                                    "attachment.content": {
                                        "query": "can I cast Hunter's Mark on an entity who is Invisible?"
                                    }
                                }
                            }
                        })

In [ ]:
rslt.json()

In [ ]:
rslt2 = elastic_request(url="_ingest/pipeline/attachment", method=requests.put, data={
  "description" : "Extract attachment information",
  "processors" : [
    {
      "attachment" : {
        "field" : "data",
        "remove_binary": True
      }
    }
  ]
})

In [ ]:
rslt2.json()

In [7]:
dir_path = "/data/Player's Handbook 5e (2014)"
encoded_files = []
for filename in os.listdir(dir_path):
    file_path = os.path.join(dir_path, filename)

    if os.path.isfile(file_path):
        with open(file_path, 'rb') as file:
            file_content = file.read()
            encoded_content = base64.b64encode(file_content).decode('utf-8')
            encoded_files.append((filename, encoded_content))

In [ ]:
for filename, contents in encoded_files:
    rslt3 = elastic_request(method=requests.put, url=f"players-handbook/_doc/{filename}?pipeline=attachment", data={"data": contents})
    print(rslt3.json())